In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import json
import csv
import requests
import lxml
import re

In [3]:
def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return zip(a, a)


## 下載普通适水平測試字表


In [ ]:
def word_one_url_generator():
    template = "http://www.pthxx.com/b_audio/03_zici_1/%02d.html"
    max_pg =66
    for i in range(1, max_pg+1):
        yield template % i
    
def word_two_url_generator():
    template = "http://www.pthxx.com/b_audio/04_zici_2/%03d.html"
    max_pg =105
    for i in range(1, max_pg+1):
        yield template % i

In [ ]:
def extract_word_list(url):
    result = requests.get(url)
    c= result.content
    soup = BeautifulSoup((c), "lxml")

    soup.prettify()

    summary = soup.find_all('div',attrs = {'id':re.compile(r'main[0-9]')})
    tables = []
    for div in summary:
        tables_ = div.find_all('table')
        tables.extend(tables_)

    data =[]

    for table in tables:
        #print('debug', table)
        rows = table.findAll('tr')
        for row in table.findAll('tr')[0:]:
            for word,pinyin in pairwise(row.findAll('td')[1:]):
                data.append((word.text.replace(' ',''), pinyin.text.replace(' ','')))

    return data

In [ ]:
word_one_gen = word_one_url_generator()

all_data = []
for url in word_one_gen:
    page_data = extract_word_list(url)
    print('debug', url, 'vocab: ', len(page_data))
    all_data.extend(page_data)
    
print(all_data)

In [ ]:
with open('./word_one.txt', 'w', newline='', encoding='UTF-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for pair in all_data:
        wr.writerow(pair)

In [ ]:
word_two_gen = word_two_url_generator()

all_data = []
for url in word_two_gen:
    page_data = extract_word_list(url)
    print('debug', url, 'vocab: ', len(page_data))
    all_data.extend(page_data)
    
print(all_data)

In [ ]:
page_data = extract_word_list(url)
print('debug', url, 'vocab: ', len(page_data))
all_data.extend(page_data)

In [ ]:
for url in word_two_gen:
    page_data = extract_word_list(url)
    print('debug', url, 'vocab: ', len(page_data))
    all_data.extend(page_data)

In [ ]:
with open('./word_two.txt', 'w', newline='', encoding='UTF-8') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    for pair in all_data:
        wr.writerow(pair)

## 讀取字表與文章

In [4]:
import pandas as pd

# Read the CSV into a pandas data frame (df)
#   With a df you can do many things
#   most important: visualize data with Seaborn
df = pd.read_csv('./word_one.txt', delimiter=',')

# Or export it in many ways, e.g. a list of tuples
word_1_list = [tuple(x) for x in df.values]

In [5]:
df = pd.read_csv('./word_two.txt', delimiter=',')

# Or export it in many ways, e.g. a list of tuples
word_2_list = [tuple(x) for x in df.values]

In [93]:
passages = []

for i in range(1,61):
    with open('./passage/%03d.txt' % i, encoding='UTF-8') as f:
        content = f.readlines()
    content = [x.strip() for x in content] 
    ann_list = []
    for word,pinyin in pairwise(content):
        ann_list.append((word, pinyin))
    passages.append(ann_list)

## 提取表1,2字的拼音

In [7]:
a_list = ['a', 'ā', 'á', 'ǎ', 'à']
o_list = ['o','ō', 'ó', 'ǒ', 'ò']
e_list = ['e', 'ē', 'é', 'ě', 'è']
i_list = ['i', 'ī', 'í', 'ǐ', 'ì']
u_list = ['u', 'ū', 'ú', 'ǔ', 'ù']
yu_list = ['ü', 'ǖ', 'ǘ', 'ǚ', 'ǜ']

consonant = ['b', 'p', 'm', 'f', 'd', 't', 'n', 'l', 'g', 'k', 'h', 'j', 'q', 'x', 'zh', 'ch', 'sh', 'r', 'z', 'c', 's', 'y', 'w']
sep = '’'
vowel = ['a', 'o', 'e', 'ai', 'ei', 'ao', 'ou', 'an', 'en', 'ang', 'eng', 'ong', 'i', 'ia', 'ie', 'iao', 'iou',
         'ian', 'in', 'iang', 'ing', 'iong', 'u', 'ua', 'uo', 'uai', 'uei', 'uan', 'uen', 'uang', 'ueng', 
         'ü', 'üe', 'üan', 'ün',
         'ue',
         'iu', 'ui', 'un' 
        ]

In [8]:
def remove_tone(alpha):
    if alpha in a_list:
        return 'a'
    elif alpha in o_list:
        return 'o'
    elif alpha in e_list:
        return 'e'
    elif alpha in i_list:
        return 'i'
    elif alpha in u_list:
        return 'u'
    elif alpha in yu_list:
        return 'ü'
    else:
        return alpha

def word_sep(word, pinyin):
    pinyin_t = ''.join([remove_tone(p) for p in pinyin])
    pinyin_len = len(pinyin)
    #print('debug', pinyin)
    result = []
    check_cnt= 0
    for w in word:
        const_p = ""
        vowel_p = ""
        
        if pinyin_t.startswith(sep) or pinyin_t.startswith("•") or pinyin_t.startswith(" "):
            pinyin_t = pinyin_t[1:]
            pinyin = pinyin[1:]
            check_cnt = check_cnt + 1
            
        for c in consonant:
            if(pinyin_t.startswith(c)):
                if(len(const_p) < len(c)):
                    const_p = c
                    
        const_p = pinyin[:len(const_p)]
        pinyin_t = pinyin_t[len(const_p):]
        pinyin = pinyin[len(const_p):]
        
        for v in vowel:
            if(pinyin_t.startswith(v)):
                #print('debugh xx', v, pinyin_t)
                if(len(vowel_p) < len(v)):
                    vowel_p = v
                    
        if len(const_p) == 0 and vowel_p == "e":
            if(pinyin_t.startswith("er")):
                vowel_p = pinyin[0:2]
        
        vowel_p = pinyin[:len(vowel_p)]
        pinyin_t = pinyin_t[len(vowel_p):]
        pinyin = pinyin[len(vowel_p):]
        #print("debug V", vowel_p, pinyin)
        result.append((w, const_p+vowel_p))
        check_cnt = check_cnt + len(const_p) + len(vowel_p)
    
    if check_cnt != pinyin_len:
        raise Exception("not all pinyin is used. %d %d" % (check_cnt, pinyin_len) + str(result) )
        
    return result
    
def character_tone(pinyin):
    '''
    pass back 1, 2, 3, 4 or 0
    '''
    for c in pinyin:
        a_idx = a_list.index(c) if c in a_list else -1
        o_idx = o_list.index(c) if c in o_list else -1
        e_idx = e_list.index(c) if c in e_list else -1
        i_idx = i_list.index(c) if c in i_list else -1
        u_idx = u_list.index(c) if c in u_list else -1
        yu_idx = yu_list.index(c) if c in yu_list else -1
        
        if(a_idx > 0):
            return a_idx
        elif(o_idx > 0):
            return o_idx
        elif(e_idx > 0):
            return e_idx
        elif(i_idx > 0):
            return i_idx
        elif(u_idx > 0):
            return u_idx
        elif(yu_idx > 0):
            return yu_idx
        
    return 0

def create_character_pinyin_dict(word_list):
    '''
    dictionary lookup by word, then each sound have a list of words.
    '''
    char_dict = {}
    for word in word_list:
        for char, pinyin in word:
            if not char in char_dict:
                pinyin_dict = {}
                pinyin_dict[pinyin] = [word]
                char_dict[char] = pinyin_dict
            else:
                pinyin_dict = char_dict[char]
                if pinyin in pinyin_dict:
                    pinyin_dict[pinyin].append(word)
                else:
                    pinyin_dict[pinyin] = [word]
                    
    return char_dict

def create_pinyin_character_dict(word_list):
    '''
    dictionary lookup by pinyin, then each character have a list of words.
    '''
    pinyin_dict = {}
    for word in word_list:
        for char, pinyin in word:
            if not pinyin in pinyin_dict:
                char_dict = {}
                char_dict[char] = [word]
                pinyin_dict[pinyin] = char_dict
            else:
                char_dict = pinyin_dict[pinyin]
                if char in char_dict:
                    char_dict[char].append(word)
                else:
                    char_dict[char] = [word]
                    
    return pinyin_dict

In [9]:
word_1_list_sep = []

for word,pinyin in word_1_list:
    try:
        test=word_sep(word,pinyin) 
        
        if len(test) == 0:
            print("Error in processing: ", word,pinyin)
        else:
            word_1_list_sep.append(test)
    except Exception as e:
        print("Error in processing: ", word,pinyin, e)

In [10]:
word_2_list_sep = []

for word,pinyin in word_2_list:
    try:
        test=word_sep(word,pinyin) 
        if len(test) == 0:
            print("Error in processing: ", word,pinyin)
        else:
            word_2_list_sep.append(test)
    except Exception as e:
        print("Error in processing: ", word,pinyin, e)

In [11]:
total_sep_list = word_1_list_sep + word_2_list_sep
print(total_sep_list[0:10])

[[('阿', 'ā'), ('姨', 'yí')], [('挨', 'āi')], [('挨', 'ái')], [('矮', 'ǎi')], [('愛', 'ài')], [('愛', 'ài'), ('國', 'guó')], [('愛', 'ài'), ('好', 'hào')], [('愛', 'ài'), ('護', 'hù')], [('愛', 'ài'), ('情', 'qíng')], [('愛', 'ài'), ('人', 'ren')]]


In [12]:
char_dict = create_character_pinyin_dict(total_sep_list)
pinyin_dict = create_pinyin_character_dict(total_sep_list)

In [13]:
char_dict['黏']

{'nián': [[('黏', 'nián')]]}

### 多音字

In [14]:
for key, value in char_dict.items():
    pinyin_dict_t = char_dict[key]
    if len(pinyin_dict_t.keys()) > 1:
        print(key + " : " + str(list(pinyin_dict_t.keys())))

阿 : ['ā', 'ē']
挨 : ['āi', 'ái']
好 : ['hào', 'hǎo']
情 : ['qíng', 'qing']
人 : ['ren', 'rén']
靜 : ['jìng', 'jing']
心 : ['xīn', 'xin']
置 : ['zhì', 'zhi']
中 : ['zhōng', 'zhòng']
會 : ['huì', 'kuài']
巴 : ['bā', 'ba']
扒 : ['bā', 'pá']
把 : ['bǎ', 'bà']
兒 : ['r', 'ér', 'er']
白 : ['bái', 'bai']
色 : ['sè', 'shǎi']
動 : ['dòng', 'dong']
家 : ['jiā', 'jia']
公 : ['gōng', 'gong']
事 : ['shì', 'shi']
扮 : ['bàn', 'ban']
樣 : ['yàng', 'yang']
乾 : ['gàn', 'gān', 'qián']
子 : ['zǐ', 'zi']
炮 : ['bāo', 'pào', 'páo']
薄 : ['báo', 'bó', 'bò']
和 : ['hé', 'hè', 'hú', 'huó', 'huò', 'huo']
石 : ['shí', 'dàn']
道 : ['dào', 'dao']
露 : ['lù', 'lòu']
發 : ['fā', 'fà', 'fa']
炸 : ['zhà', 'zhá']
背 : ['bēi', 'bèi']
方 : ['fāng', 'fang']
奔 : ['bēn', 'bèn']
性 : ['xìng', 'xing']
重 : ['zhòng', 'zhóng', 'chóng']
要 : ['yào', 'yāo']
更 : ['gēng', 'gèng']
量 : ['liàng', 'liáng', 'liang']
便 : ['biàn', 'pián']
別 : ['bié', 'biè']
達 : ['dá', 'da']
餅 : ['bǐng', 'bing']
屏 : ['bǐng', 'píng']
長 : ['cháng', 'zhǎng']
剝 : ['bō', 'bāo']
削 : ['xuē', 'xiā

### 同音字查詢

In [15]:
def query_same_sound(char):
    p_dict = char_dict[char]
    for key in p_dict.keys():
        print(key + " : " + str(list(pinyin_dict[key].keys())))
        
query_same_sound("職")

zhí : ['值', '直', '職', '殖', '執', '植', '侄']


### 拼音查詢

In [16]:
a_list = ['a', 'ā', 'á', 'ǎ', 'à']
o_list = ['o','ō', 'ó', 'ǒ', 'ò']
e_list = ['e', 'ē', 'é', 'ě', 'è']
i_list = ['i', 'ī', 'í', 'ǐ', 'ì']
u_list = ['u', 'ū', 'ú', 'ǔ', 'ù']
yu_list = ['ü', 'ǖ', 'ǘ', 'ǚ', 'ǜ']

In [17]:
pinyin_dict["rě"]

{'惹': [[('惹', 'rě')]]}

## 文章朗讀難點

In [18]:
from IPython.display import Markdown, display

def printmd(string, color=None): 
    colorstr = "<span style='color:{}'>{}</span>".format(color, string) 
    display(Markdown(colorstr)) 
    
def formatmd(string, color=None): 
    colorstr = "<span style='color:{}'>{}</span>".format(color, string) 
    return colorstr

In [18]:
import thulac

thu1 = thulac.thulac()  #默认模式
text = thu1.cut("我爱北京天安门", text=True)  #进行一句话分词
print(text)

ModuleNotFoundError: No module named 'thulac'

### 文章分詞

In [124]:
passage_no = 50

passage_str = ""
for py, ch in passages[passage_no-1]:
    if not ch == "//": 
        passage_str = passage_str + ch
    
print(passage_str) 
text = thu1.cut(passage_str, text=True)  #进行一句话分词
print(text)

著名教育家班傑明曾經接到一個青年人的求救電話，並與那個向往成功、渴望指點的青年人約好了見面的時間和地點。待那個青年如約而至時，班傑明的房門敞開着，眼前的景象卻令青年人頗感意外——班傑明的房間裏亂七八糟、狼藉一片。沒等青年人開口，班傑明就招呼道： “你看我這房間，太不整潔了，請你在門外等候一分鐘，我收拾一下，你再進來吧。 ”一邊說着，班傑明就輕輕地關上了房門。不到一分鐘的時間，班傑明就又打開了房門並熱情地把青年人讓進客廳。這時，青年人的眼前展現出另一番景象——房間內的一切已變得井然有序，而且有兩杯剛剛倒好的紅酒，在淡淡的香水氣息裏還漾着微波。可是，沒等青年人把滿腹的有關人生和事業的疑難問題向班傑明講出來，班傑明就非常客氣地說道： “乾杯。你可以走了。 ”青年人手持酒杯一下子愣住了，既尷尬又非常遺憾地說： “可是，我……我還沒向您請教呢…… ”“這些……難道還不夠嗎？ ”班傑明一邊微笑着一邊掃視着自己的房間，輕言細語地說， “你進來又有一分鐘了。 ”“一分鐘……一分鐘…… ”青年人若有所思地說： “我懂了，您讓我明白了一分鐘的時間可以做許多事情，可以改變許多事情的深刻道理。 ”班傑明舒心地笑了。青年人把杯裏的紅酒一飲而盡，向班傑明連連道謝後，開心地走了。其實，只要把握好生命的每一分鐘，也就把握了理想的人生。


NameError: name 'thu1' is not defined

In [43]:
import jieba

seg_list = jieba.cut(passage_str, cut_all=False)
seg_list = list(seg_list)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

ImportError: No module named 'jieba'

### 主要詞彙

In [27]:
from itertools import groupby
from collections import Counter

counter=Counter(seg_list)
freq_lst=[list(counter.keys()),list(counter.values())]
freq_df = pd.DataFrame(np.array(freq_lst).T, columns=['Letter','wordCount'])
freq_df["wordCount"] = freq_df["wordCount"].astype(int)
sort_df = freq_df.sort_values("wordCount", ascending=False)
print(sort_df[sort_df["wordCount"] > 1])

NameError: name 'seg_list' is not defined

### 必讀輕聲

In [118]:
def softly_sound(passage):
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 0: 
            print(ch, py)
            
#softly_sound(passages[passage_no-1])

def highlight_softly_sound(passage):
    result = ""
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 0: 
            result += formatmd(ch, "blue")
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_softly_sound(passages[passage_no-1])

那是力爭上游<span style='color:blue'>的</span>一種樹，筆直<span style='color:blue'>的</span>幹，筆直<span style='color:blue'>的</span>枝。它<span style='color:blue'>的</span>幹<span style='color:blue'>呢</span>，通常是丈把高，像是加以人工似<span style='color:blue'>的</span>，一丈以內，絕無旁枝；它所有<span style='color:blue'>的</span>椏枝<span style='color:blue'>呢</span>，一律向上，而且緊緊靠攏，也像是加以人工似<span style='color:blue'>的</span>，成為一束，絕無橫斜逸出；它<span style='color:blue'>的</span>寬大<span style='color:blue'>的</span>葉<span style='color:blue'>子</span>也是片片向上，幾乎沒有斜生<span style='color:blue'>的</span>，更不用說倒垂<span style='color:blue'>了</span>；它<span style='color:blue'>的</span>皮，光滑而有銀色<span style='color:blue'>的</span>暈圈，微微泛出淡青色。這是雖在北方<span style='color:blue'>的</span>風雪<span style='color:blue'>的</span>壓迫下卻保持<span style='color:blue'>着</span>倔強挺立<span style='color:blue'>的</span>一種樹！哪怕只有碗來粗細<span style='color:blue'>罷</span>，它卻努力向上發展，高到丈許，兩丈，參天聳立，不折不撓，對抗<span style='color:blue'>着</span>西北風。這就是白楊樹，西北極普通<span style='color:blue'>的</span>一種樹，然而決不是平凡<span style='color:blue'>的</span>樹！它沒有婆娑<span style='color:blue'>的</span>姿態，沒有屈曲盤旋<span style='color:blue'>的</span>虯枝，也許你要說它不美麗， ──如果美是專指“婆娑”或“橫斜逸出”之類而言，那<span style='color:blue'>麼</span>，白楊樹算不得樹中<span style='color:blue'>的</span>好女子；但是它卻是偉岸，正直，樸質，嚴肅，也不缺乏溫和，更不用提它<span style='color:blue'>的</span>堅強不屈與挺拔，它是樹中<span style='color:blue'>的</span>偉丈夫！當你在積雪初融<span style='color:blue'>的</span>高原上走過，看見平坦<span style='color:blue'>的</span>大地上傲然挺立這<span style='color:blue'>麼</span>一株或一排白楊樹，難道你就只覺<span style='color:blue'>得</span>樹只是樹，難道你就不想到它<span style='color:blue'>的</span>樸質，嚴肅，堅強不屈，至少也象徵<span style='color:blue'>了</span>北方<span style='color:blue'>的</span>農民；難道你竟一點兒也不聯想到，在敵後<span style='color:blue'>的</span>廣大土//地上，到處有堅強不屈，就像這白楊樹一樣傲然挺立<span style='color:blue'>的</span>守衛他<span style='color:blue'>們</span>家鄉<span style='color:blue'>的</span>哨兵！難道你又不更遠一點想到這樣枝枝葉葉靠緊團結，力求上進<span style='color:blue'>的</span>白楊樹，宛然象徵<span style='color:blue'>了</span>今天在華北平原縱橫決蕩用血寫出新中國歷史<span style='color:blue'>的</span>那種精<span style='color:blue'>神</span>和意志。

### 去聲變調

In [119]:
def third_sound_steak(passage):
    cur = ""
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 3: 
            cur += ch
        else:
            if len(cur) > 1:
                print(cur)
            cur = ""
            
#third_sound_steak(passages[passage_no-1])

def highlight_third_sound_steak(passage):
    result = ""
    cur = ""
    for py, ch in passage:
        if len(py.strip()) > 0 and character_tone(py) == 3: 
            cur += ch
        else:
            if len(cur) > 1:
                result += formatmd(cur, 'blue')
            cur = ""
            result += formatmd(ch)
            
    display(Markdown(result)) 
            
highlight_third_sound_steak(passages[passage_no-1])

<span style='color:None'>那</span><span style='color:None'>是</span><span style='color:None'>力</span><span style='color:None'>爭</span><span style='color:None'>上</span><span style='color:None'>游</span><span style='color:None'>的</span><span style='color:None'>一</span><span style='color:None'>樹</span><span style='color:None'>，</span><span style='color:None'>直</span><span style='color:None'>的</span><span style='color:None'>幹</span><span style='color:None'>，</span><span style='color:None'>直</span><span style='color:None'>的</span><span style='color:None'>枝</span><span style='color:None'>。</span><span style='color:None'>它</span><span style='color:None'>的</span><span style='color:None'>幹</span><span style='color:None'>呢</span><span style='color:None'>，</span><span style='color:None'>通</span><span style='color:None'>常</span><span style='color:None'>是</span><span style='color:None'>丈</span><span style='color:None'>高</span><span style='color:None'>，</span><span style='color:None'>像</span><span style='color:None'>是</span><span style='color:None'>加</span><span style='color:None'>人</span><span style='color:None'>工</span><span style='color:None'>似</span><span style='color:None'>的</span><span style='color:None'>，</span><span style='color:None'>一</span><span style='color:None'>丈</span><span style='color:None'>內</span><span style='color:None'>，</span><span style='color:None'>絕</span><span style='color:None'>無</span><span style='color:None'>旁</span><span style='color:None'>枝</span><span style='color:None'>；</span><span style='color:None'>它</span><span style='color:blue'>所有</span><span style='color:None'>的</span><span style='color:None'>椏</span><span style='color:None'>枝</span><span style='color:None'>呢</span><span style='color:None'>，</span><span style='color:None'>一</span><span style='color:None'>律</span><span style='color:None'>向</span><span style='color:None'>上</span><span style='color:None'>，</span><span style='color:None'>而</span><span style='color:blue'>且緊緊</span><span style='color:None'>靠</span><span style='color:None'>，</span><span style='color:None'>像</span><span style='color:None'>是</span><span style='color:None'>加</span><span style='color:None'>人</span><span style='color:None'>工</span><span style='color:None'>似</span><span style='color:None'>的</span><span style='color:None'>，</span><span style='color:None'>成</span><span style='color:None'>為</span><span style='color:None'>一</span><span style='color:None'>束</span><span style='color:None'>，</span><span style='color:None'>絕</span><span style='color:None'>無</span><span style='color:None'>橫</span><span style='color:None'>斜</span><span style='color:None'>逸</span><span style='color:None'>出</span><span style='color:None'>；</span><span style='color:None'>它</span><span style='color:None'>的</span><span style='color:None'>寬</span><span style='color:None'>大</span><span style='color:None'>的</span><span style='color:None'>葉</span><span style='color:None'>子</span><span style='color:None'>是</span><span style='color:None'>片</span><span style='color:None'>片</span><span style='color:None'>向</span><span style='color:None'>上</span><span style='color:None'>，</span><span style='color:None'>幾</span><span style='color:None'>乎</span><span style='color:None'>沒</span><span style='color:None'>斜</span><span style='color:None'>生</span><span style='color:None'>的</span><span style='color:None'>，</span><span style='color:None'>更</span><span style='color:None'>不</span><span style='color:None'>用</span><span style='color:None'>說</span><span style='color:None'>倒</span><span style='color:None'>垂</span><span style='color:None'>了</span><span style='color:None'>；</span><span style='color:None'>它</span><span style='color:None'>的</span><span style='color:None'>皮</span><span style='color:None'>，</span><span style='color:None'>光</span><span style='color:None'>滑</span><span style='color:None'>而</span><span style='color:None'>銀</span><span style='color:None'>色</span><span style='color:None'>的</span><span style='color:None'>暈</span><span style='color:None'>圈</span><span style='color:None'>，</span><span style='color:None'>微</span><span style='color:None'>微</span><span style='color:None'>泛</span><span style='color:None'>出</span><span style='color:None'>淡</span><span style='color:None'>青</span><span style='color:None'>色</span><span style='color:None'>。</span><span style='color:None'>這</span><span style='color:None'>是</span><span style='color:None'>雖</span><span style='color:None'>在</span><span style='color:None'>方</span><span style='color:None'>的</span><span style='color:None'>風</span><span style='color:None'>的</span><span style='color:None'>壓</span><span style='color:None'>迫</span><span style='color:None'>下</span><span style='color:None'>卻</span><span style='color:None'>持</span><span style='color:None'>着</span><span style='color:None'>倔</span><span style='color:None'>強</span><span style='color:None'>立</span><span style='color:None'>的</span><span style='color:None'>一</span><span style='color:None'>樹</span><span style='color:None'>！</span><span style='color:None'>怕</span><span style='color:blue'>只有碗</span><span style='color:None'>來</span><span style='color:None'>粗</span><span style='color:None'>細</span><span style='color:None'>罷</span><span style='color:None'>，</span><span style='color:None'>它</span><span style='color:None'>卻</span><span style='color:None'>力</span><span style='color:None'>向</span><span style='color:None'>上</span><span style='color:None'>發</span><span style='color:None'>，</span><span style='color:None'>高</span><span style='color:None'>到</span><span style='color:None'>丈</span><span style='color:None'>，</span><span style='color:None'>丈</span><span style='color:None'>，</span><span style='color:None'>參</span><span style='color:None'>天</span><span style='color:None'>立</span><span style='color:None'>，</span><span style='color:None'>不</span><span style='color:None'>折</span><span style='color:None'>不</span><span style='color:None'>撓</span><span style='color:None'>，</span><span style='color:None'>對</span><span style='color:None'>抗</span><span style='color:None'>着</span><span style='color:None'>西</span><span style='color:None'>風</span><span style='color:None'>。</span><span style='color:None'>這</span><span style='color:None'>就</span><span style='color:None'>是</span><span style='color:None'>白</span><span style='color:None'>楊</span><span style='color:None'>樹</span><span style='color:None'>，</span><span style='color:None'>西</span><span style='color:None'>極</span><span style='color:None'>通</span><span style='color:None'>的</span><span style='color:None'>一</span><span style='color:None'>樹</span><span style='color:None'>，</span><span style='color:None'>然</span><span style='color:None'>而</span><span style='color:None'>決</span><span style='color:None'>不</span><span style='color:None'>是</span><span style='color:None'>平</span><span style='color:None'>凡</span><span style='color:None'>的</span><span style='color:None'>樹</span><span style='color:None'>！</span><span style='color:None'>它</span><span style='color:None'>沒</span><span style='color:None'>婆</span><span style='color:None'>娑</span><span style='color:None'>的</span><span style='color:None'>姿</span><span style='color:None'>態</span><span style='color:None'>，</span><span style='color:None'>沒</span><span style='color:None'>屈</span><span style='color:None'>曲</span><span style='color:None'>盤</span><span style='color:None'>旋</span><span style='color:None'>的</span><span style='color:None'>虯</span><span style='color:None'>枝</span><span style='color:None'>，</span><span style='color:blue'>也許你</span><span style='color:None'>要</span><span style='color:None'>說</span><span style='color:None'>它</span><span style='color:None'>不</span><span style='color:None'>麗</span><span style='color:None'>， ──</span><span style='color:None'>如</span><span style='color:blue'>果美</span><span style='color:None'>是</span><span style='color:None'>專</span><span style='color:None'>“</span><span style='color:None'>婆</span><span style='color:None'>娑</span><span style='color:None'>”</span><span style='color:None'>或</span><span style='color:None'>“</span><span style='color:None'>橫</span><span style='color:None'>斜</span><span style='color:None'>逸</span><span style='color:None'>出</span><span style='color:None'>”</span><span style='color:None'>之</span><span style='color:None'>類</span><span style='color:None'>而</span><span style='color:None'>言</span><span style='color:None'>，</span><span style='color:None'>那</span><span style='color:None'>麼</span><span style='color:None'>，</span><span style='color:None'>白</span><span style='color:None'>楊</span><span style='color:None'>樹</span><span style='color:None'>算</span><span style='color:None'>不</span><span style='color:None'>得</span><span style='color:None'>樹</span><span style='color:None'>中</span><span style='color:None'>的</span><span style='color:blue'>好女子</span><span style='color:None'>；</span><span style='color:None'>但</span><span style='color:None'>是</span><span style='color:None'>它</span><span style='color:None'>卻</span><span style='color:None'>是</span><span style='color:None'>岸</span><span style='color:None'>，</span><span style='color:None'>正</span><span style='color:None'>直</span><span style='color:None'>，</span><span style='color:None'>質</span><span style='color:None'>，</span><span style='color:None'>嚴</span><span style='color:None'>肅</span><span style='color:None'>，</span><span style='color:None'>不</span><span style='color:None'>缺</span><span style='color:None'>乏</span><span style='color:None'>溫</span><span style='color:None'>和</span><span style='color:None'>，</span><span style='color:None'>更</span><span style='color:None'>不</span><span style='color:None'>用</span><span style='color:None'>提</span><span style='color:None'>它</span><span style='color:None'>的</span><span style='color:None'>堅</span><span style='color:None'>強</span><span style='color:None'>不</span><span style='color:None'>屈</span><span style='color:blue'>與挺</span><span style='color:None'>拔</span><span style='color:None'>，</span><span style='color:None'>它</span><span style='color:None'>是</span><span style='color:None'>樹</span><span style='color:None'>中</span><span style='color:None'>的</span><span style='color:None'>丈</span><span style='color:None'>夫</span><span style='color:None'>！</span><span style='color:None'>當</span><span style='color:None'>在</span><span style='color:None'>積</span><span style='color:None'>初</span><span style='color:None'>融</span><span style='color:None'>的</span><span style='color:None'>高</span><span style='color:None'>原</span><span style='color:None'>上</span><span style='color:None'>過</span><span style='color:None'>，</span><span style='color:None'>看</span><span style='color:None'>見</span><span style='color:None'>平</span><span style='color:None'>的</span><span style='color:None'>大</span><span style='color:None'>地</span><span style='color:None'>上</span><span style='color:None'>傲</span><span style='color:None'>然</span><span style='color:None'>立</span><span style='color:None'>這</span><span style='color:None'>麼</span><span style='color:None'>一</span><span style='color:None'>株</span><span style='color:None'>或</span><span style='color:None'>一</span><span style='color:None'>排</span><span style='color:None'>白</span><span style='color:None'>楊</span><span style='color:None'>樹</span><span style='color:None'>，</span><span style='color:None'>難</span><span style='color:None'>道</span><span style='color:None'>就</span><span style='color:None'>覺</span><span style='color:None'>得</span><span style='color:None'>樹</span><span style='color:None'>是</span><span style='color:None'>樹</span><span style='color:None'>，</span><span style='color:None'>難</span><span style='color:None'>道</span><span style='color:None'>就</span><span style='color:None'>不</span><span style='color:None'>到</span><span style='color:None'>它</span><span style='color:None'>的</span><span style='color:None'>質</span><span style='color:None'>，</span><span style='color:None'>嚴</span><span style='color:None'>肅</span><span style='color:None'>，</span><span style='color:None'>堅</span><span style='color:None'>強</span><span style='color:None'>不</span><span style='color:None'>屈</span><span style='color:None'>，</span><span style='color:None'>至</span><span style='color:blue'>少也</span><span style='color:None'>象</span><span style='color:None'>徵</span><span style='color:None'>了</span><span style='color:None'>方</span><span style='color:None'>的</span><span style='color:None'>農</span><span style='color:None'>民</span><span style='color:None'>；</span><span style='color:None'>難</span><span style='color:None'>道</span><span style='color:None'>竟</span><span style='color:None'>一</span><span style='color:blue'>點兒也</span><span style='color:None'>不</span><span style='color:None'>聯</span><span style='color:None'>到</span><span style='color:None'>，</span><span style='color:None'>在</span><span style='color:None'>敵</span><span style='color:None'>後</span><span style='color:None'>的</span><span style='color:None'>大</span><span style='color:None'>//</span><span style='color:None'>地</span><span style='color:None'>上</span><span style='color:None'>，</span><span style='color:None'>到</span><span style='color:None'>處</span><span style='color:None'>堅</span><span style='color:None'>強</span><span style='color:None'>不</span><span style='color:None'>屈</span><span style='color:None'>，</span><span style='color:None'>就</span><span style='color:None'>像</span><span style='color:None'>這</span><span style='color:None'>白</span><span style='color:None'>楊</span><span style='color:None'>樹</span><span style='color:None'>一</span><span style='color:None'>樣</span><span style='color:None'>傲</span><span style='color:None'>然</span><span style='color:None'>立</span><span style='color:None'>的</span><span style='color:None'>衛</span><span style='color:None'>他</span><span style='color:None'>們</span><span style='color:None'>家</span><span style='color:None'>鄉</span><span style='color:None'>的</span><span style='color:None'>哨</span><span style='color:None'>兵</span><span style='color:None'>！</span><span style='color:None'>難</span><span style='color:None'>道</span><span style='color:None'>又</span><span style='color:None'>不</span><span style='color:None'>更</span><span style='color:None'>一</span><span style='color:blue'>點想</span><span style='color:None'>到</span><span style='color:None'>這</span><span style='color:None'>樣</span><span style='color:None'>枝</span><span style='color:None'>枝</span><span style='color:None'>葉</span><span style='color:None'>葉</span><span style='color:None'>靠</span><span style='color:None'>團</span><span style='color:None'>結</span><span style='color:None'>，</span><span style='color:None'>力</span><span style='color:None'>求</span><span style='color:None'>上</span><span style='color:None'>進</span><span style='color:None'>的</span><span style='color:None'>白</span><span style='color:None'>楊</span><span style='color:None'>樹</span><span style='color:None'>，</span><span style='color:None'>然</span><span style='color:None'>象</span><span style='color:None'>徵</span><span style='color:None'>了</span><span style='color:None'>今</span><span style='color:None'>天</span><span style='color:None'>在</span><span style='color:None'>華</span><span style='color:None'>平</span><span style='color:None'>原</span><span style='color:None'>縱</span><span style='color:None'>橫</span><span style='color:None'>決</span><span style='color:None'>蕩</span><span style='color:None'>用</span><span style='color:None'>血</span><span style='color:None'>出</span><span style='color:None'>新</span><span style='color:None'>中</span><span style='color:None'>國</span><span style='color:None'>歷</span><span style='color:None'>的</span><span style='color:None'>那</span><span style='color:None'>精</span><span style='color:None'>神</span><span style='color:None'>和</span><span style='color:None'>意</span><span style='color:None'>志</span><span style='color:None'>。</span>

### 一不變調

In [120]:
def highlight_yi_bu(passage):
    result = ""
    cur = ""
    for py, ch in passage:
        if (ch == '一' and py != 'yì') or (ch == '不' and py != 'bù'):
            result += formatmd(ch, 'blue')
        elif (ch == '一' and py == 'yì') or (ch == '不' and py == 'bù'):
            result += formatmd(ch, 'green')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_yi_bu(passages[passage_no-1])

那是力爭上游的<span style='color:green'>一</span>種樹，筆直的幹，筆直的枝。它的幹呢，通常是丈把高，像是加以人工似的，<span style='color:blue'>一</span>丈以內，絕無旁枝；它所有的椏枝呢，<span style='color:blue'>一</span>律向上，而且緊緊靠攏，也像是加以人工似的，成為<span style='color:blue'>一</span>束，絕無橫斜逸出；它的寬大的葉子也是片片向上，幾乎沒有斜生的，更<span style='color:blue'>不</span>用說倒垂了；它的皮，光滑而有銀色的暈圈，微微泛出淡青色。這是雖在北方的風雪的壓迫下卻保持着倔強挺立的<span style='color:green'>一</span>種樹！哪怕只有碗來粗細罷，它卻努力向上發展，高到丈許，兩丈，參天聳立，<span style='color:green'>不</span>折<span style='color:green'>不</span>撓，對抗着西北風。這就是白楊樹，西北極普通的<span style='color:green'>一</span>種樹，然而決<span style='color:blue'>不</span>是平凡的樹！它沒有婆娑的姿態，沒有屈曲盤旋的虯枝，也許你要說它<span style='color:green'>不</span>美麗， ──如果美是專指“婆娑”或“橫斜逸出”之類而言，那麼，白楊樹算<span style='color:green'>不</span>得樹中的好女子；但是它卻是偉岸，正直，樸質，嚴肅，也<span style='color:green'>不</span>缺乏溫和，更<span style='color:blue'>不</span>用提它的堅強<span style='color:green'>不</span>屈與挺拔，它是樹中的偉丈夫！當你在積雪初融的高原上走過，看見平坦的大地上傲然挺立這麼<span style='color:green'>一</span>株或<span style='color:green'>一</span>排白楊樹，難道你就只覺得樹只是樹，難道你就<span style='color:green'>不</span>想到它的樸質，嚴肅，堅強<span style='color:green'>不</span>屈，至少也象徵了北方的農民；難道你竟<span style='color:green'>一</span>點兒也<span style='color:green'>不</span>聯想到，在敵後的廣大土//地上，到處有堅強<span style='color:green'>不</span>屈，就像這白楊樹<span style='color:blue'>一</span>樣傲然挺立的守衛他們家鄉的哨兵！難道你又<span style='color:blue'>不</span>更遠<span style='color:green'>一</span>點想到這樣枝枝葉葉靠緊團結，力求上進的白楊樹，宛然象徵了今天在華北平原縱橫決蕩用血寫出新中國歷史的那種精神和意志。

In [ ]:
### 平舌音

In [121]:
def zcs_in_passage(passage): 
    for py, ch in passage:
        if((py.startswith("z") or py.startswith("c") or py.startswith("s"))
          and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            print(ch, py)
            
#zcs_in_passage(passages[passage_no-1])

def highlight_zcs_in_passage(passage):
    result = ""
    for py, ch in passage:
        if((py.startswith("z") or py.startswith("c") or py.startswith("s"))
          and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_zcs_in_passage(passages[passage_no-1])


那是力爭上游的一種樹，筆直的幹，筆直的枝。它的幹呢，通常是丈把高，像是加以人工似的，一丈以內，絕無旁枝；它<span style='color:blue'>所</span>有的椏枝呢，一律向上，而且緊緊靠攏，也像是加以人工似的，成為一束，絕無橫斜逸出；它的寬大的葉<span style='color:blue'>子</span>也是片片向上，幾乎沒有斜生的，更不用說倒垂了；它的皮，光滑而有銀<span style='color:blue'>色</span>的暈圈，微微泛出淡青<span style='color:blue'>色</span>。這是<span style='color:blue'>雖</span><span style='color:blue'>在</span>北方的風雪的壓迫下卻保持着倔強挺立的一種樹！哪怕只有碗來<span style='color:blue'>粗</span>細罷，它卻努力向上發展，高到丈許，兩丈，<span style='color:blue'>參</span>天<span style='color:blue'>聳</span>立，不折不撓，對抗着西北風。這就是白楊樹，西北極普通的一種樹，然而決不是平凡的樹！它沒有婆<span style='color:blue'>娑</span>的<span style='color:blue'>姿</span>態，沒有屈曲盤旋的虯枝，也許你要說它不美麗， ──如果美是專指“婆<span style='color:blue'>娑</span>”或“橫斜逸出”之類而言，那麼，白楊樹<span style='color:blue'>算</span>不得樹中的好女<span style='color:blue'>子</span>；但是它卻是偉岸，正直，樸質，嚴<span style='color:blue'>肅</span>，也不缺乏溫和，更不用提它的堅強不屈與挺拔，它是樹中的偉丈夫！當你<span style='color:blue'>在</span>積雪初融的高原上<span style='color:blue'>走</span>過，看見平坦的大地上傲然挺立這麼一株或一排白楊樹，難道你就只覺得樹只是樹，難道你就不想到它的樸質，嚴<span style='color:blue'>肅</span>，堅強不屈，至少也象徵了北方的農民；難道你竟一點兒也不聯想到，<span style='color:blue'>在</span>敵後的廣大土//地上，到處有堅強不屈，就像這白楊樹一樣傲然挺立的守衛他們家鄉的哨兵！難道你又不更遠一點想到這樣枝枝葉葉靠緊團結，力求上進的白楊樹，宛然象徵了今天<span style='color:blue'>在</span>華北平原<span style='color:blue'>縱</span>橫決蕩用血寫出新中國歷史的那種精神和意志。

### 翹舌音

In [122]:
def zhchsh_in_passage(passage): 
    for py, ch in passage:
        if py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            print(ch, py)
            
#zhchsh_in_passage(passages[passage_no-1])

def highlight_zhchsh_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_zhchsh_in_passage(passages[passage_no-1])

那<span style='color:blue'>是</span>力<span style='color:blue'>爭</span><span style='color:blue'>上</span>游的一<span style='color:blue'>種</span><span style='color:blue'>樹</span>，筆<span style='color:blue'>直</span>的幹，筆<span style='color:blue'>直</span>的<span style='color:blue'>枝</span>。它的幹呢，通<span style='color:blue'>常</span><span style='color:blue'>是</span><span style='color:blue'>丈</span>把高，像<span style='color:blue'>是</span>加以人工<span style='color:blue'>似</span>的，一<span style='color:blue'>丈</span>以內，絕無旁<span style='color:blue'>枝</span>；它所有的椏<span style='color:blue'>枝</span>呢，一律向<span style='color:blue'>上</span>，而且緊緊靠攏，也像<span style='color:blue'>是</span>加以人工<span style='color:blue'>似</span>的，<span style='color:blue'>成</span>為一<span style='color:blue'>束</span>，絕無橫斜逸<span style='color:blue'>出</span>；它的寬大的葉子也<span style='color:blue'>是</span>片片向<span style='color:blue'>上</span>，幾乎沒有斜<span style='color:blue'>生</span>的，更不用<span style='color:blue'>說</span>倒<span style='color:blue'>垂</span>了；它的皮，光滑而有銀色的暈圈，微微泛<span style='color:blue'>出</span>淡青色。<span style='color:blue'>這</span><span style='color:blue'>是</span>雖在北方的風雪的壓迫下卻保<span style='color:blue'>持</span><span style='color:blue'>着</span>倔強挺立的一<span style='color:blue'>種</span><span style='color:blue'>樹</span>！哪怕<span style='color:blue'>只</span>有碗來粗細罷，它卻努力向<span style='color:blue'>上</span>發<span style='color:blue'>展</span>，高到<span style='color:blue'>丈</span>許，兩<span style='color:blue'>丈</span>，參天聳立，不<span style='color:blue'>折</span>不撓，對抗<span style='color:blue'>着</span>西北風。<span style='color:blue'>這</span>就<span style='color:blue'>是</span>白楊<span style='color:blue'>樹</span>，西北極普通的一<span style='color:blue'>種</span><span style='color:blue'>樹</span>，然而決不<span style='color:blue'>是</span>平凡的<span style='color:blue'>樹</span>！它沒有婆娑的姿態，沒有屈曲盤旋的虯<span style='color:blue'>枝</span>，也許你要<span style='color:blue'>說</span>它不美麗， ──如果美<span style='color:blue'>是</span><span style='color:blue'>專</span><span style='color:blue'>指</span>“婆娑”或“橫斜逸<span style='color:blue'>出</span>”<span style='color:blue'>之</span>類而言，那麼，白楊<span style='color:blue'>樹</span>算不得<span style='color:blue'>樹</span><span style='color:blue'>中</span>的好女子；但<span style='color:blue'>是</span>它卻<span style='color:blue'>是</span>偉岸，<span style='color:blue'>正</span><span style='color:blue'>直</span>，樸<span style='color:blue'>質</span>，嚴肅，也不缺乏溫和，更不用提它的堅強不屈與挺拔，它<span style='color:blue'>是</span><span style='color:blue'>樹</span><span style='color:blue'>中</span>的偉<span style='color:blue'>丈</span>夫！當你在積雪<span style='color:blue'>初</span>融的高原<span style='color:blue'>上</span>走過，看見平坦的大地<span style='color:blue'>上</span>傲然挺立<span style='color:blue'>這</span>麼一<span style='color:blue'>株</span>或一排白楊<span style='color:blue'>樹</span>，難道你就<span style='color:blue'>只</span>覺得<span style='color:blue'>樹</span><span style='color:blue'>只</span><span style='color:blue'>是</span><span style='color:blue'>樹</span>，難道你就不想到它的樸<span style='color:blue'>質</span>，嚴肅，堅強不屈，<span style='color:blue'>至</span><span style='color:blue'>少</span>也象<span style='color:blue'>徵</span>了北方的農民；難道你竟一點兒也不聯想到，在敵後的廣大土//地<span style='color:blue'>上</span>，到<span style='color:blue'>處</span>有堅強不屈，就像<span style='color:blue'>這</span>白楊<span style='color:blue'>樹</span>一樣傲然挺立的<span style='color:blue'>守</span>衛他們家鄉的<span style='color:blue'>哨</span>兵！難道你又不更遠一點想到<span style='color:blue'>這</span>樣<span style='color:blue'>枝</span><span style='color:blue'>枝</span>葉葉靠緊團結，力求<span style='color:blue'>上</span>進的白楊<span style='color:blue'>樹</span>，宛然象<span style='color:blue'>徵</span>了今天在華北平原縱橫決蕩用血寫<span style='color:blue'>出</span>新<span style='color:blue'>中</span>國歷<span style='color:blue'>史</span>的那<span style='color:blue'>種</span>精<span style='color:blue'>神</span>和意<span style='color:blue'>志</span>。

In [123]:
def highlight_zcs_zhchsh_in_passage(passage):
    result = ""
    for py, ch in passage:
        if ((py.startswith("z") or py.startswith("c") or py.startswith("s"))
          and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            result += formatmd(ch, 'blue')
        elif py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            result += formatmd(ch, 'green')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_zcs_zhchsh_in_passage(passages[passage_no-1])

那<span style='color:green'>是</span>力<span style='color:green'>爭</span><span style='color:green'>上</span>游的一<span style='color:green'>種</span><span style='color:green'>樹</span>，筆<span style='color:green'>直</span>的幹，筆<span style='color:green'>直</span>的<span style='color:green'>枝</span>。它的幹呢，通<span style='color:green'>常</span><span style='color:green'>是</span><span style='color:green'>丈</span>把高，像<span style='color:green'>是</span>加以人工<span style='color:green'>似</span>的，一<span style='color:green'>丈</span>以內，絕無旁<span style='color:green'>枝</span>；它<span style='color:blue'>所</span>有的椏<span style='color:green'>枝</span>呢，一律向<span style='color:green'>上</span>，而且緊緊靠攏，也像<span style='color:green'>是</span>加以人工<span style='color:green'>似</span>的，<span style='color:green'>成</span>為一<span style='color:green'>束</span>，絕無橫斜逸<span style='color:green'>出</span>；它的寬大的葉<span style='color:blue'>子</span>也<span style='color:green'>是</span>片片向<span style='color:green'>上</span>，幾乎沒有斜<span style='color:green'>生</span>的，更不用<span style='color:green'>說</span>倒<span style='color:green'>垂</span>了；它的皮，光滑而有銀<span style='color:blue'>色</span>的暈圈，微微泛<span style='color:green'>出</span>淡青<span style='color:blue'>色</span>。<span style='color:green'>這</span><span style='color:green'>是</span><span style='color:blue'>雖</span><span style='color:blue'>在</span>北方的風雪的壓迫下卻保<span style='color:green'>持</span><span style='color:green'>着</span>倔強挺立的一<span style='color:green'>種</span><span style='color:green'>樹</span>！哪怕<span style='color:green'>只</span>有碗來<span style='color:blue'>粗</span>細罷，它卻努力向<span style='color:green'>上</span>發<span style='color:green'>展</span>，高到<span style='color:green'>丈</span>許，兩<span style='color:green'>丈</span>，<span style='color:blue'>參</span>天<span style='color:blue'>聳</span>立，不<span style='color:green'>折</span>不撓，對抗<span style='color:green'>着</span>西北風。<span style='color:green'>這</span>就<span style='color:green'>是</span>白楊<span style='color:green'>樹</span>，西北極普通的一<span style='color:green'>種</span><span style='color:green'>樹</span>，然而決不<span style='color:green'>是</span>平凡的<span style='color:green'>樹</span>！它沒有婆<span style='color:blue'>娑</span>的<span style='color:blue'>姿</span>態，沒有屈曲盤旋的虯<span style='color:green'>枝</span>，也許你要<span style='color:green'>說</span>它不美麗， ──如果美<span style='color:green'>是</span><span style='color:green'>專</span><span style='color:green'>指</span>“婆<span style='color:blue'>娑</span>”或“橫斜逸<span style='color:green'>出</span>”<span style='color:green'>之</span>類而言，那麼，白楊<span style='color:green'>樹</span><span style='color:blue'>算</span>不得<span style='color:green'>樹</span><span style='color:green'>中</span>的好女<span style='color:blue'>子</span>；但<span style='color:green'>是</span>它卻<span style='color:green'>是</span>偉岸，<span style='color:green'>正</span><span style='color:green'>直</span>，樸<span style='color:green'>質</span>，嚴<span style='color:blue'>肅</span>，也不缺乏溫和，更不用提它的堅強不屈與挺拔，它<span style='color:green'>是</span><span style='color:green'>樹</span><span style='color:green'>中</span>的偉<span style='color:green'>丈</span>夫！當你<span style='color:blue'>在</span>積雪<span style='color:green'>初</span>融的高原<span style='color:green'>上</span><span style='color:blue'>走</span>過，看見平坦的大地<span style='color:green'>上</span>傲然挺立<span style='color:green'>這</span>麼一<span style='color:green'>株</span>或一排白楊<span style='color:green'>樹</span>，難道你就<span style='color:green'>只</span>覺得<span style='color:green'>樹</span><span style='color:green'>只</span><span style='color:green'>是</span><span style='color:green'>樹</span>，難道你就不想到它的樸<span style='color:green'>質</span>，嚴<span style='color:blue'>肅</span>，堅強不屈，<span style='color:green'>至</span><span style='color:green'>少</span>也象<span style='color:green'>徵</span>了北方的農民；難道你竟一點兒也不聯想到，<span style='color:blue'>在</span>敵後的廣大土//地<span style='color:green'>上</span>，到<span style='color:green'>處</span>有堅強不屈，就像<span style='color:green'>這</span>白楊<span style='color:green'>樹</span>一樣傲然挺立的<span style='color:green'>守</span>衛他們家鄉的<span style='color:green'>哨</span>兵！難道你又不更遠一點想到<span style='color:green'>這</span>樣<span style='color:green'>枝</span><span style='color:green'>枝</span>葉葉靠緊團結，力求<span style='color:green'>上</span>進的白楊<span style='color:green'>樹</span>，宛然象<span style='color:green'>徵</span>了今天<span style='color:blue'>在</span>華北平原<span style='color:blue'>縱</span>橫決蕩用血寫<span style='color:green'>出</span>新<span style='color:green'>中</span>國歷<span style='color:green'>史</span>的那<span style='color:green'>種</span>精<span style='color:green'>神</span>和意<span style='color:green'>志</span>。

### 多音字

In [114]:
def multiple_sound_in_passage(passage):
    for py, ch in passage:
        if ch in char_dict:
            p_dict = char_dict[ch]
            if len(p_dict.keys()) > 1:
                print(ch, py, list(p_dict.keys()))

# multiple_sound_in_passage(passages[passage_no-1])

def highlight_multiple_sound_in_passage(passage):
    result = ""
    for py, ch in passage:
        if ch in char_dict:
            p_dict = char_dict[ch]
            valid_cnt = 0
            for key in p_dict.keys():
                if character_tone(key) >0:
                    valid_cnt = valid_cnt+1
            
            if valid_cnt > 1:
                result += formatmd(ch, 'red')
            else:
                result += ch
            
    display(Markdown(result)) 
            
highlight_multiple_sound_in_passage(passages[passage_no-1])

兩個同齡<span style='color:red'>的</span>年輕人同時受於一家店且拿同樣<span style='color:red'>的</span>薪水可是一段時<span style='color:red'>間</span>後叫<span style='color:red'>阿</span>諾德<span style='color:red'>的</span>那個小伙子青雲直<span style='color:red'>上</span>而那個叫布魯諾<span style='color:red'>的</span>小伙子卻仍在原地踏步布魯諾很不滿意老闆<span style='color:red'>的</span>不公<span style='color:red'>正</span><span style='color:red'>待</span>遇終於有一天他到老闆<span style='color:red'>發</span>牢騷了老闆一邊耐心地聽他<span style='color:red'>的</span>抱怨一邊在心裏盤算怎樣向他<span style='color:red'>解</span>釋清楚他<span style='color:red'>和</span><span style='color:red'>阿</span>諾德之<span style='color:red'>間</span><span style='color:red'>的</span><span style='color:red'>差</span><span style='color:red'>別</span>布魯諾先生老闆開口說話了您現在到集市<span style='color:red'>上</span>去一下<span style='color:red'>看</span><span style='color:red'>看</span>今天早<span style='color:red'>上</span>有<span style='color:red'>什</span>麼賣<span style='color:red'>的</span>布魯諾從集市<span style='color:red'>上</span>回來向老闆匯報說今早集市<span style='color:red'>上</span>只有一個農民<span style='color:red'>拉</span>了一<span style='color:red'>車</span>土豆在賣有多<span style='color:red'>少</span>老闆問布魯諾趕快戴<span style='color:red'>上</span>帽子又跑到集<span style='color:red'>上</span>然後回來告訴老闆一共四十袋土豆價格是多<span style='color:red'>少</span>布魯諾又第三次跑到集<span style='color:red'>上</span>問來了價格<span style='color:red'>好</span>吧老闆對他說現在請您坐到這<span style='color:red'>把</span>椅子<span style='color:red'>上</span>一句話也不<span style='color:red'>要</span>說<span style='color:red'>看</span><span style='color:red'>看</span><span style='color:red'>阿</span>諾德怎麼說<span style='color:red'>阿</span>諾德很快就從集市<span style='color:red'>上</span>回來了向老闆匯報說到現在<span style='color:red'>為</span>止只有一個農民在賣土豆一共四十口袋價格是多<span style='color:red'>少</span>多<span style='color:red'>少</span>土豆質<span style='color:red'>量</span>很不錯他帶回來一個讓老闆<span style='color:red'>看</span><span style='color:red'>看</span>這個農民一個鐘頭以後<span style='color:red'>還</span><span style='color:red'>會</span>弄來<span style='color:red'>幾</span>箱西紅柿據他<span style='color:red'>看</span>價格非常公道昨天他們子<span style='color:red'>的</span>西紅柿賣<span style='color:red'>得</span>很快庫存已經不多了他想這麼<span style='color:red'>便</span>宜<span style='color:red'>的</span>西紅柿老闆肯定<span style='color:red'>會</span><span style='color:red'>要</span>進一些<span style='color:red'>的</span>所以他不僅帶回了一個西紅柿做樣品而且<span style='color:red'>把</span>那個農民也帶來了他現在<span style='color:red'>正</span>在外面等回話呢此時老闆<span style='color:red'>轉</span>向了布魯諾說現在您肯定知道<span style='color:red'>為</span><span style='color:red'>什</span>麼<span style='color:red'>阿</span>諾德<span style='color:red'>的</span>薪水比您高了吧

In [30]:
char_dict['切']

{'qiè': [[('密', 'mì'), ('切', 'qiè')],
  [('迫', 'pò'), ('切', 'qiè')],
  [('切', 'qiè'), ('實', 'shí')],
  [('親', 'qīn'), ('切', 'qiè')],
  [('確', 'què'), ('切', 'qiè')],
  [('一', 'yī'), ('切', 'qiè')],
  [('懇', 'kěn'), ('切', 'qiè')],
  [('切', 'qiè'), ('合', 'hé')],
  [('切', 'qiè'), ('忌', 'jì')],
  [('切', 'qiè'), ('身', 'shēn')],
  [('熱', 'rè'), ('切', 'qiè')],
  [('深', 'shēn'), ('切', 'qiè')],
  [('殷', 'yīn'), ('切', 'qiè')],
  [('真', 'zhēn'), ('切', 'qiè')]],
 'qiē': [[('切', 'qiē')],
  [('切', 'qiē')],
  [('關', 'guān'), ('切', 'qiē')],
  [('急', 'jí'), ('切', 'qiē')],
  [('切', 'qiē'), ('除', 'chú')],
  [('切', 'qiē'), ('磋', 'cuō')],
  [('切', 'qiē'), ('點', 'diǎn')],
  [('切', 'qiē'), ('割', 'gē')],
  [('切', 'qiē'), ('口', 'kǒu')],
  [('切', 'qiē'), ('麵', 'miàn')],
  [('切', 'qiē'), ('片', 'piàn')],
  [('切', 'qiē'), ('綫', 'xiàn')],
  [('貼', 'tiē'), ('切', 'qiē')],
  [('心', 'xīn'), ('切', 'qiē')]]}

### 兒化音

In [115]:
def r_in_passage(passage): 
    for py, ch in passage:
        if py.endswith("r"):
            print(ch, py)
            
#r_in_passage(passages[passage_no-1])

def highlight_r_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("r"):
            result += formatmd(ch, 'red')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_r_in_passage(passages[passage_no-1])

兩個同齡的年輕人同時受僱於一家店舖，並且拿同樣的薪水。可是一段時間後，叫阿諾德的那個小伙子青雲直上，<span style='color:red'>而</span>那個叫布魯諾的小伙子卻仍在原地踏步。布魯諾很不滿意老闆的不公正待遇。終於有一天他到老闆<span style='color:red'>那兒</span>發牢騷了。老闆一邊耐心地聽着他的抱怨，一邊在心裏盤算着怎樣向他解釋清楚他和阿諾德之間的差別。“布魯諾先生， ”老闆開口說話了， “您現在到集市上去一下，看看今天早上有什麼賣的。 ”布魯諾從集市上回來向老闆匯報說，今早集市上只有一個農民拉了一車土豆在賣。“有多少？ ”老闆問。布魯諾趕快戴上帽子又跑到集上，然後回來告訴老闆一共四十袋土豆。“價格是多少？ ”布魯諾又第三次跑到集上問來了價格。好吧， ”老闆對他說， “現在請您坐到這把椅子上一句話也不要說，看看阿諾德怎麼說。 ”阿諾德很快就從集市上回來了。向老闆匯報說到現在為止只有一個農民在賣土豆，一共四十口袋，價格是多少多少；土豆質量很不錯，他帶回來一個讓老闆看看。這個農民一個鐘頭以後還會弄來幾箱西紅柿，據他看價格非常公道。昨天他們舖子的西紅柿賣得很快，庫存已經不//多了。他想這麼便宜的西紅柿，老闆肯定會要進一些的，所以他不僅帶回了一個西紅柿做樣品，<span style='color:red'>而</span>且把那個農民也帶來了，他現在正在外面等回話呢。此時老闆轉向了布魯諾，說： “現在您肯定知道為什麼阿諾德的薪水比您高了吧！ ”

### 前鼻音

In [28]:
def n_in_passage(passage): 
    for py, ch in passage:
        if py.endswith("n"):
            print(ch, py)
            
#n_in_passage(passages[passage_no-1])

def highlight_n_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("n"):
            result += formatmd(ch, 'red')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_n_in_passage(passages[passage_no-1])

我在加拿大學習期<span style='color:red'>間</span>遇到過兩次募<span style='color:red'>捐</span>，那情景至<span style='color:red'>今</span>使我<span style='color:red'>難</span>以忘懷。一<span style='color:red'>天</span>，我在渥太華的街上被兩個<span style='color:red'>男</span>孩子<span style='color:red'>攔</span>住去路，他<span style='color:red'>們</span>十來歲，<span style='color:red'>穿</span>得整整齊齊，每<span style='color:red'>人</span>頭上戴着個做工精巧、色彩<span style='color:red'>鮮</span><span style='color:red'>豔</span>的紙帽，上<span style='color:red'>面</span>寫着“為幫助<span style='color:red'>患</span>小兒麻痹的夥<span style='color:red'>伴</span>募<span style='color:red'>捐</span>” 。其中的一個，不由<span style='color:red'>分</span>說就坐在小凳上給我擦起皮鞋來，另一個則<span style='color:red'>彬</span><span style='color:red'>彬</span>有禮地發<span style='color:red'>問</span>： “小姐，<span style='color:red'>您</span>是哪國<span style='color:red'>人</span>？喜<span style='color:red'>歡</span>渥太華嗎？ ” “小姐，在你<span style='color:red'>們</span>國家有沒有小孩兒<span style='color:red'>患</span>小兒麻痹？誰給他<span style='color:red'>們</span>醫療費？ ”一<span style='color:red'>連</span><span style='color:red'>串</span>的<span style='color:red'>問</span>題，使我這個有生以來頭一次在眾目睽睽之下讓別<span style='color:red'>人</span>擦鞋的異鄉<span style='color:red'>人</span>，從<span style='color:red'>近</span>乎狼狽的窘態中解脫出來。我<span style='color:red'>們</span>像朋友一樣聊起天兒來……幾個月之後，也是在街上。一些十字路口處或車<span style='color:red'>站</span>坐着幾位老<span style='color:red'>人</span>。他<span style='color:red'>們</span><span style='color:red'>滿</span>頭<span style='color:red'>銀</span>髮，<span style='color:red'>身</span><span style='color:red'>穿</span>各種老式<span style='color:red'>軍</span>裝，上<span style='color:red'>面</span>佈<span style='color:red'>滿</span>了大大小小形形色色的徽章、獎章，每<span style='color:red'>人</span>手捧一大束<span style='color:red'>鮮</span>花，有水<span style='color:red'>仙</span>、石竹、玫瑰及叫不出名字的，一色雪白。匆匆過往的行<span style='color:red'>人</span><span style='color:red'>紛</span><span style='color:red'>紛</span>止步，把<span style='color:red'>錢</span>投<span style='color:red'>進</span>這些老<span style='color:red'>人</span><span style='color:red'>身</span>旁的白色木箱內，<span style='color:red'>然</span>後向他<span style='color:red'>們</span>微微鞠躬，從他<span style='color:red'>們</span>手中接過一朵花。我<span style='color:red'>看</span>了一會兒，有<span style='color:red'>人</span>投一兩<span style='color:red'>元</span>，有<span style='color:red'>人</span>投幾百<span style='color:red'>元</span>，還有<span style='color:red'>人</span>掏出支票<span style='color:red'>填</span>好後投<span style='color:red'>進</span>木箱。那些老<span style='color:red'>軍</span><span style='color:red'>人</span>毫不注意<span style='color:red'>人</span><span style='color:red'>們</span><span style='color:red'>捐</span>多少<span style='color:red'>錢</span>，一直不//停地向<span style='color:red'>人</span><span style='color:red'>們</span>低聲道謝。同行的朋友告訴我，這是為紀<span style='color:red'>念</span>二次大<span style='color:red'>戰</span>中<span style='color:red'>參</span><span style='color:red'>戰</span>的勇士，募<span style='color:red'>捐</span>救濟<span style='color:red'>殘</span>廢<span style='color:red'>軍</span><span style='color:red'>人</span>和烈士遺孀，每<span style='color:red'>年</span>一次；<span style='color:red'>認</span><span style='color:red'>捐</span>的<span style='color:red'>人</span>可謂踴躍，而且秩序井<span style='color:red'>然</span>，氣<span style='color:red'>氛</span>莊<span style='color:red'>嚴</span>。有些地方，<span style='color:red'>人</span><span style='color:red'>們</span>還耐<span style='color:red'>心</span>地排着隊。我想，這是<span style='color:red'>因</span>為他<span style='color:red'>們</span>都知道：正是這些老<span style='color:red'>人</span><span style='color:red'>們</span>的流血犧牲<span style='color:red'>換</span>來了包括他<span style='color:red'>們</span><span style='color:red'>信</span>仰自由在內的許許多多。我兩次把那微不足道的一點兒<span style='color:red'>錢</span>捧給他<span style='color:red'>們</span>，只想對他<span style='color:red'>們</span>說聲“謝謝” 。

### 後鼻音

In [29]:
def ng_in_passage(passage): 
    for py, ch in passage:
        if py.endswith("ng"):
            print(ch, py)
            
#ng_in_passage(passages[passage_no-1])

def highlight_ng_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("ng"):
            result += formatmd(ch, 'red')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_ng_in_passage(passages[passage_no-1])

我在加拿大學習期間遇到過<span style='color:red'>兩</span>次募捐，那<span style='color:red'>情</span><span style='color:red'>景</span>至今使我難以<span style='color:red'>忘</span>懷。一天，我在渥太華的街<span style='color:red'>上</span>被<span style='color:red'>兩</span>個男孩子攔住去路，他們十來歲，穿得<span style='color:red'>整</span><span style='color:red'>整</span>齊齊，每人頭<span style='color:red'>上</span>戴着個做<span style='color:red'>工</span><span style='color:red'>精</span>巧、色彩鮮豔的紙帽，<span style='color:red'>上</span>面寫着“為<span style='color:red'>幫</span>助患小兒麻痹的夥伴募捐” 。其<span style='color:red'>中</span>的一個，不由分說就坐在小<span style='color:red'>凳</span><span style='color:red'>上</span>給我擦起皮鞋來，<span style='color:red'>另</span>一個則彬彬有禮地發問： “小姐，您是哪國人？喜歡渥太華嗎？ ” “小姐，在你們國家有沒有小孩兒患小兒麻痹？誰給他們醫療費？ ”一連串的問題，使我這個有<span style='color:red'>生</span>以來頭一次在<span style='color:red'>眾</span>目睽睽之下<span style='color:red'>讓</span>別人擦鞋的異<span style='color:red'>鄉</span>人，<span style='color:red'>從</span>近乎<span style='color:red'>狼</span>狽的<span style='color:red'>窘</span>態<span style='color:red'>中</span>解脫出來。我們<span style='color:red'>像</span><span style='color:red'>朋</span>友一<span style='color:red'>樣</span>聊起天兒來……幾個月之後，也是在街<span style='color:red'>上</span>。一些十字路口處或車站坐着幾位老人。他們滿頭銀髮，身穿各<span style='color:red'>種</span>老式軍<span style='color:red'>裝</span>，<span style='color:red'>上</span>面佈滿了大大小小<span style='color:red'>形</span><span style='color:red'>形</span>色色的徽<span style='color:red'>章</span>、<span style='color:red'>獎</span><span style='color:red'>章</span>，每人手<span style='color:red'>捧</span>一大束鮮花，有水仙、石竹、玫瑰及叫不出<span style='color:red'>名</span>字的，一色雪白。<span style='color:red'>匆</span><span style='color:red'>匆</span>過<span style='color:red'>往</span>的<span style='color:red'>行</span>人紛紛止步，把錢投進這些老人身<span style='color:red'>旁</span>的白色木<span style='color:red'>箱</span>內，然後<span style='color:red'>向</span>他們微微鞠<span style='color:red'>躬</span>，<span style='color:red'>從</span>他們手<span style='color:red'>中</span>接過一朵花。我看了一會兒，有人投一<span style='color:red'>兩</span>元，有人投幾百元，還有人掏出支票填好後投進木<span style='color:red'>箱</span>。那些老軍人毫不注意人們捐多少錢，一直不//<span style='color:red'>停</span>地<span style='color:red'>向</span>人們低<span style='color:red'>聲</span>道謝。<span style='color:red'>同</span><span style='color:red'>行</span>的<span style='color:red'>朋</span>友告訴我，這是為紀念二次大戰<span style='color:red'>中</span>參戰的<span style='color:red'>勇</span>士，募捐救濟殘廢軍人和烈士遺<span style='color:red'>孀</span>，每年一次；認捐的人可謂<span style='color:red'>踴</span>躍，而且秩序<span style='color:red'>井</span>然，氣氛<span style='color:red'>莊</span>嚴。有些地<span style='color:red'>方</span>，人們還耐心地排着隊。我<span style='color:red'>想</span>，這是因為他們都知道：<span style='color:red'>正</span>是這些老人們的流血犧<span style='color:red'>牲</span>換來了包括他們信<span style='color:red'>仰</span>自由在內的許許多多。我<span style='color:red'>兩</span>次把那微不足道的一點兒錢<span style='color:red'>捧</span>給他們，只<span style='color:red'>想</span>對他們說<span style='color:red'>聲</span>“謝謝” 。

In [30]:
def highlight_g_ng_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.endswith("n"):
            result += formatmd(ch, 'green')
        elif py.endswith("ng"):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_g_ng_in_passage(passages[passage_no-1])

我在加拿大學習期<span style='color:green'>間</span>遇到過<span style='color:blue'>兩</span>次募<span style='color:green'>捐</span>，那<span style='color:blue'>情</span><span style='color:blue'>景</span>至<span style='color:green'>今</span>使我<span style='color:green'>難</span>以<span style='color:blue'>忘</span>懷。一<span style='color:green'>天</span>，我在渥太華的街<span style='color:blue'>上</span>被<span style='color:blue'>兩</span>個<span style='color:green'>男</span>孩子<span style='color:green'>攔</span>住去路，他<span style='color:green'>們</span>十來歲，<span style='color:green'>穿</span>得<span style='color:blue'>整</span><span style='color:blue'>整</span>齊齊，每<span style='color:green'>人</span>頭<span style='color:blue'>上</span>戴着個做<span style='color:blue'>工</span><span style='color:blue'>精</span>巧、色彩<span style='color:green'>鮮</span><span style='color:green'>豔</span>的紙帽，<span style='color:blue'>上</span><span style='color:green'>面</span>寫着“為<span style='color:blue'>幫</span>助<span style='color:green'>患</span>小兒麻痹的夥<span style='color:green'>伴</span>募<span style='color:green'>捐</span>” 。其<span style='color:blue'>中</span>的一個，不由<span style='color:green'>分</span>說就坐在小<span style='color:blue'>凳</span><span style='color:blue'>上</span>給我擦起皮鞋來，<span style='color:blue'>另</span>一個則<span style='color:green'>彬</span><span style='color:green'>彬</span>有禮地發<span style='color:green'>問</span>： “小姐，<span style='color:green'>您</span>是哪國<span style='color:green'>人</span>？喜<span style='color:green'>歡</span>渥太華嗎？ ” “小姐，在你<span style='color:green'>們</span>國家有沒有小孩兒<span style='color:green'>患</span>小兒麻痹？誰給他<span style='color:green'>們</span>醫療費？ ”一<span style='color:green'>連</span><span style='color:green'>串</span>的<span style='color:green'>問</span>題，使我這個有<span style='color:blue'>生</span>以來頭一次在<span style='color:blue'>眾</span>目睽睽之下<span style='color:blue'>讓</span>別<span style='color:green'>人</span>擦鞋的異<span style='color:blue'>鄉</span><span style='color:green'>人</span>，<span style='color:blue'>從</span><span style='color:green'>近</span>乎<span style='color:blue'>狼</span>狽的<span style='color:blue'>窘</span>態<span style='color:blue'>中</span>解脫出來。我<span style='color:green'>們</span><span style='color:blue'>像</span><span style='color:blue'>朋</span>友一<span style='color:blue'>樣</span>聊起天兒來……幾個月之後，也是在街<span style='color:blue'>上</span>。一些十字路口處或車<span style='color:green'>站</span>坐着幾位老<span style='color:green'>人</span>。他<span style='color:green'>們</span><span style='color:green'>滿</span>頭<span style='color:green'>銀</span>髮，<span style='color:green'>身</span><span style='color:green'>穿</span>各<span style='color:blue'>種</span>老式<span style='color:green'>軍</span><span style='color:blue'>裝</span>，<span style='color:blue'>上</span><span style='color:green'>面</span>佈<span style='color:green'>滿</span>了大大小小<span style='color:blue'>形</span><span style='color:blue'>形</span>色色的徽<span style='color:blue'>章</span>、<span style='color:blue'>獎</span><span style='color:blue'>章</span>，每<span style='color:green'>人</span>手<span style='color:blue'>捧</span>一大束<span style='color:green'>鮮</span>花，有水<span style='color:green'>仙</span>、石竹、玫瑰及叫不出<span style='color:blue'>名</span>字的，一色雪白。<span style='color:blue'>匆</span><span style='color:blue'>匆</span>過<span style='color:blue'>往</span>的<span style='color:blue'>行</span><span style='color:green'>人</span><span style='color:green'>紛</span><span style='color:green'>紛</span>止步，把<span style='color:green'>錢</span>投<span style='color:green'>進</span>這些老<span style='color:green'>人</span><span style='color:green'>身</span><span style='color:blue'>旁</span>的白色木<span style='color:blue'>箱</span>內，<span style='color:green'>然</span>後<span style='color:blue'>向</span>他<span style='color:green'>們</span>微微鞠<span style='color:blue'>躬</span>，<span style='color:blue'>從</span>他<span style='color:green'>們</span>手<span style='color:blue'>中</span>接過一朵花。我<span style='color:green'>看</span>了一會兒，有<span style='color:green'>人</span>投一<span style='color:blue'>兩</span><span style='color:green'>元</span>，有<span style='color:green'>人</span>投幾百<span style='color:green'>元</span>，還有<span style='color:green'>人</span>掏出支票<span style='color:green'>填</span>好後投<span style='color:green'>進</span>木<span style='color:blue'>箱</span>。那些老<span style='color:green'>軍</span><span style='color:green'>人</span>毫不注意<span style='color:green'>人</span><span style='color:green'>們</span><span style='color:green'>捐</span>多少<span style='color:green'>錢</span>，一直不//<span style='color:blue'>停</span>地<span style='color:blue'>向</span><span style='color:green'>人</span><span style='color:green'>們</span>低<span style='color:blue'>聲</span>道謝。<span style='color:blue'>同</span><span style='color:blue'>行</span>的<span style='color:blue'>朋</span>友告訴我，這是為紀<span style='color:green'>念</span>二次大<span style='color:green'>戰</span><span style='color:blue'>中</span><span style='color:green'>參</span><span style='color:green'>戰</span>的<span style='color:blue'>勇</span>士，募<span style='color:green'>捐</span>救濟<span style='color:green'>殘</span>廢<span style='color:green'>軍</span><span style='color:green'>人</span>和烈士遺<span style='color:blue'>孀</span>，每<span style='color:green'>年</span>一次；<span style='color:green'>認</span><span style='color:green'>捐</span>的<span style='color:green'>人</span>可謂<span style='color:blue'>踴</span>躍，而且秩序<span style='color:blue'>井</span><span style='color:green'>然</span>，氣<span style='color:green'>氛</span><span style='color:blue'>莊</span><span style='color:green'>嚴</span>。有些地<span style='color:blue'>方</span>，<span style='color:green'>人</span><span style='color:green'>們</span>還耐<span style='color:green'>心</span>地排着隊。我<span style='color:blue'>想</span>，這是<span style='color:green'>因</span>為他<span style='color:green'>們</span>都知道：<span style='color:blue'>正</span>是這些老<span style='color:green'>人</span><span style='color:green'>們</span>的流血犧<span style='color:blue'>牲</span><span style='color:green'>換</span>來了包括他<span style='color:green'>們</span><span style='color:green'>信</span><span style='color:blue'>仰</span>自由在內的許許多多。我<span style='color:blue'>兩</span>次把那微不足道的一點兒<span style='color:green'>錢</span><span style='color:blue'>捧</span>給他<span style='color:green'>們</span>，只<span style='color:blue'>想</span>對他<span style='color:green'>們</span>說<span style='color:blue'>聲</span>“謝謝” 。

### n/l 音分別

In [31]:
def highlight_n_l_in_passage(passage):
    result = ""
    for py, ch in passage:
        if py.startswith("n"):
            result += formatmd(ch, 'green')
        elif py.startswith("l"):
            result += formatmd(ch, 'blue')
        else:
            result += ch
            
    display(Markdown(result)) 
            
highlight_n_l_in_passage(passages[passage_no-1])

我在加<span style='color:green'>拿</span>大學習期間遇到過<span style='color:blue'>兩</span>次募捐，<span style='color:green'>那</span>情景至今使我<span style='color:green'>難</span>以忘懷。一天，我在渥太華的街上被<span style='color:blue'>兩</span>個<span style='color:green'>男</span>孩子<span style='color:blue'>攔</span>住去<span style='color:blue'>路</span>，他們十<span style='color:blue'>來</span>歲，穿得整整齊齊，每人頭上戴着個做工精巧、色彩鮮豔的紙帽，上面寫着“為幫助患小兒麻痹的夥伴募捐” 。其中的一個，不由分說就坐在小凳上給我擦起皮鞋<span style='color:blue'>來</span>，<span style='color:blue'>另</span>一個則彬彬有<span style='color:blue'>禮</span>地發問： “小姐，<span style='color:green'>您</span>是<span style='color:green'>哪</span>國人？喜歡渥太華嗎？ ” “小姐，在<span style='color:green'>你</span>們國家有沒有小孩兒患小兒麻痹？誰給他們醫<span style='color:blue'>療</span>費？ ”一<span style='color:blue'>連</span>串的問題，使我這個有生以<span style='color:blue'>來</span>頭一次在眾目睽睽之下讓別人擦鞋的異鄉人，從近乎<span style='color:blue'>狼</span>狽的窘態中解脫出<span style='color:blue'>來</span>。我們像朋友一樣<span style='color:blue'>聊</span>起天兒<span style='color:blue'>來</span>……幾個月之後，也是在街上。一些十字<span style='color:blue'>路</span>口處或車站坐着幾位<span style='color:blue'>老</span>人。他們滿頭銀髮，身穿各種<span style='color:blue'>老</span>式軍裝，上面佈滿<span style='color:blue'>了</span>大大小小形形色色的徽章、獎章，每人手捧一大束鮮花，有水仙、石竹、玫瑰及叫不出名字的，一色雪白。匆匆過往的行人紛紛止步，把錢投進這些<span style='color:blue'>老</span>人身旁的白色木箱<span style='color:green'>內</span>，然後向他們微微鞠躬，從他們手中接過一朵花。我看<span style='color:blue'>了</span>一會兒，有人投一<span style='color:blue'>兩</span>元，有人投幾百元，還有人掏出支票填好後投進木箱。<span style='color:green'>那</span>些<span style='color:blue'>老</span>軍人毫不注意人們捐多少錢，一直不//停地向人們低聲道謝。同行的朋友告訴我，這是為紀<span style='color:green'>念</span>二次大戰中參戰的勇士，募捐救濟殘廢軍人和<span style='color:blue'>烈</span>士遺孀，每<span style='color:green'>年</span>一次；認捐的人可謂踴躍，而且秩序井然，氣氛莊嚴。有些地方，人們還<span style='color:green'>耐</span>心地排着隊。我想，這是因為他們都知道：正是這些<span style='color:blue'>老</span>人們的<span style='color:blue'>流</span>血犧牲換<span style='color:blue'>來</span><span style='color:blue'>了</span>包括他們信仰自由在<span style='color:green'>內</span>的許許多多。我<span style='color:blue'>兩</span>次把<span style='color:green'>那</span>微不足道的一點兒錢捧給他們，只想對他們說聲“謝謝” 。

### 統計

* 平舌音字數:翹舌音字數
* 前鼻音字數:後鼻音字數
* 多音字字數
* 一不變調詞數
* 去聲變調詞數
* 兒化音詞數

In [181]:
def stats_passage(passage):
    zcs_list = []
    zhchsh_list = []
    n_list = []
    ng_list = []
    n_list2 = []
    l_list = []
    
    for py, ch in passage:
        if ((py.startswith("z") or py.startswith("c") or py.startswith("s"))
              and not(py.startswith("zh") or py.startswith("ch") or py.startswith("sh"))):
            zcs_list.append(ch)
        if py.startswith("zh") or py.startswith("ch") or py.startswith("sh"):
            zhchsh_list.append(ch)
        if py.endswith("n"):
            n_list.append(ch)
        if py.endswith("ng"):
            ng_list.append(ch)
        if py.startswith("n"):
            n_list2.append(ch)
        if py.startswith("l"):
            l_list.append(ch)
        
    print("平舌音字數:翹舌音字數 = %d : %d" % (len(zcs_list), len(zhchsh_list))) 
    print("前鼻音字數:後鼻音字數 = %d : %d" % (len(n_list), len(ng_list))) 
    print("N 音字數: L 音字數 = %d : %d" % (len(n_list2), len(l_list))) 
    
stats_passage(passages[passage_no-1])

平舌音字數:翹舌音字數 = 30 : 74
前鼻音字數:後鼻音字數 = 69 : 45
N 音字數: L 音字數 = 15 : 45
